install packages

In [ ]:
!pip install gradio

: 

In [ ]:
import pickle

from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from joblib import load
import pickle
from scipy.sparse import vstack, hstack, csr_matrix
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

: 

In [ ]:
model = load('./ridge_model.pkl')


# PKLファイルから変換器を読み込みます
with open('./cv_cate.pkl', 'rb') as f:
    cv_cate = pickle.load(f)

with open('./cv_name.pkl', 'rb') as f2:
    cv_name = pickle.load(f2)

with open('./tv.pkl', 'rb') as e:
    tv = pickle.load(e)

with open('./lb.pkl', 'rb') as g:
    lb = pickle.load(g)

: 

**アプリ**

雛形

In [17]:
import gradio as gr
import pickle
import pandas as pd



# Load the pre-trained model
# with open('path/to/your/model.pkl', 'rb') as model_file:
#     model = pickle.load(model_file)

def predict_price(name, item_condition_id, category_name, brand_name, price, shipping, item_description):
    # 日本語の入力を英語に翻訳
    name_en = translate_to_english(name)
    category_name_en = translate_to_english(category_name)
    brand_name_en = translate_to_english(brand_name)
    item_description_en = translate_to_english(item_description)

    shipping=1 if shipping=="送料込み" else 0
    print(shipping)
    if item_condition_id=="全体的に状態が悪い" or "傷や汚れあり":
      item_condition_id=5
    elif item_condition_id=="やや傷や汚れあり":
      item_condition_id=4
    elif item_condition_id=="目立った傷や汚れなし":
      item_condition_id=3
    elif  item_condition_id=="未使用に近い":
      item_condition_id=2
    elif item_condition_id== "新品、未使用":
      item_conditon_id=1


    # Create a DataFrame with the translated input data
    data = {
        'name': [name_en],
        'item_condition_id': [item_condition_id],
        'category_name': [category_name_en],
        'brand_name': [brand_name_en],
        'price': [price],
        'shipping': [shipping],
        'item_description': [item_description_en]
    }
    combined = pd.DataFrame(data)

    combined = combined.astype({'item_condition_id': 'category', 'category_name': 'category', 'brand_name': 'category'})
    combined['item_condition_id'] = pd.Categorical(combined['item_condition_id'], categories=[1, 2, 3, 4, 5], ordered=True)
    X_dummies = csr_matrix(pd.get_dummies(combined[['item_condition_id', 'shipping']], sparse=True).values)
    X_name = cv_name.transform(combined['name'])
    X_category = cv_cate.transform(combined['category_name'])

    X_description = tv.transform(combined['item_description'])
    X_brand = lb.transform(combined['brand_name'])
    sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name)).tocsr()

    # Make predictions
    predictions = model.predict(sparse_merge)

    r = 100.0 - (price / (predictions*201)) * 100
    if r >= 0:
        result = "{}%お得です".format(r)
    else:
        r = -r 
        result = "{}%損です".format(r)
    # Return the prediction and the message
    predictions = predictions*201
    answer1 = "推定値段が{}円でした．．".format(predictions)
    answer2 = answer1 + result
    return answer2

# Define the Gradio interface
cond = [ "全体的に状態が悪い","傷や汚れあり","やや傷や汚れあり","目立った傷や汚れなし","未使用に近い","新品、未使用"]
cond.reverse()
interface = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.components.Textbox(label="商品名をコピペしてください"),
        gr.components.Radio(cond,label="商品の状態"),
        gr.components.Textbox(label="カテゴリー名をコピペしてください"),
        gr.components.Textbox(label="ブランド名をコピペしてください"),
        gr.components.Number(label="商品の出品金額を入力してください"),
        gr.components.Radio([ "送料込み","着払い"],label="送料"),
        gr.components.Textbox(label="商品の説明をコピペしてください")
    ],
    title="メルカリ買い得チェッカー",
    outputs=gr.components.Textbox(label="Predicted Price")
)


# Webアプリを作成
#app = gr.Interface(fn=greet_english, inputs="text", outputs="text", title="English Greeting Translator")


# Launch the interface
if __name__ == "__main__":
    interface.launch(debug=True,share=True)


preprocessing

In [ ]:
import gradio as gr
import pickle
import pandas as pd



# Load the pre-trained model
# with open('path/to/your/model.pkl', 'rb') as model_file:
#     model = pickle.load(model_file)

def predict_price(name, item_condition_id, category_name, brand_name, price, shipping, item_description):
    # 日本語の入力を英語に翻訳
    name_en = translate_to_english(name)
    category_name_en = translate_to_english(category_name)
    brand_name_en = translate_to_english(brand_name)
    item_description_en = translate_to_english(item_description)

    shipping=1 if shipping=="送料込み" else 0
    print(shipping)
    if item_condition_id=="全体的に状態が悪い" or "傷や汚れあり":
      item_condition_id=5
    elif item_condition_id=="やや傷や汚れあり":
      item_condition_id=4
    elif item_condition_id=="目立った傷や汚れなし":
      item_condition_id=3
    elif  item_condition_id=="未使用に近い":
      item_condition_id=2
    elif item_condition_id== "新品、未使用":
      item_conditon_id=1


    # Create a DataFrame with the translated input data
    data = {
        'name': [name_en],
        'item_condition_id': [item_condition_id],
        'category_name': [category_name_en],
        'brand_name': [brand_name_en],
        'price': [price],
        'shipping': [shipping],
        'item_description': [item_description_en]
    }
    combined = pd.DataFrame(data)

    combined = combined.astype({'item_condition_id': 'category', 'category_name': 'category', 'brand_name': 'category'})
    combined['item_condition_id'] = pd.Categorical(combined['item_condition_id'], categories=[1, 2, 3, 4, 5], ordered=True)
    X_dummies = csr_matrix(pd.get_dummies(combined[['item_condition_id', 'shipping']], sparse=True).values)
    X_name = cv_name.transform(combined['name'])
    X_category = cv_cate.transform(combined['category_name'])

    X_description = tv.transform(combined['item_description'])
    X_brand = lb.transform(combined['brand_name'])
    sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name)).tocsr()

    # Make predictions
    predictions = model.predict(sparse_merge)

    r = 100.0 - (price / predictions*201) * 100
    if r >= 0:
        result = f"{r:.2f}%お得です"
    else:
        result = f"{-r:.2f}%損です"
    # Return the prediction and the message
    return predictions, result

    # Return the prediction
    return predictions

# Define the Gradio interface
cond = [ "全体的に状態が悪い","傷や汚れあり","やや傷や汚れあり","目立った傷や汚れなし","未使用に近い","新品、未使用"]
cond.reverse()
interface = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.components.Textbox(label="商品名をコピペしてください"),
        gr.components.Radio(cond,label="商品の状態"),
        gr.components.Textbox(label="カテゴリー名をコピペしてください"),
        gr.components.Textbox(label="ブランド名をコピペしてください"),
        gr.components.Number(label="商品の出品金額を入力してください"),
        gr.components.Radio([ "送料込み","着払い"],label="送料"),
        gr.components.Textbox(label="商品の説明をコピペしてください")
    ],
    title="メルカリ買い得チェッカー",
    outputs=gr.components.Textbox(label="Predicted Price")
)


# Webアプリを作成
#app = gr.Interface(fn=greet_english, inputs="text", outputs="text", title="English Greeting Translator")


# Launch the interface
if __name__ == "__main__":
    interface.launch(debug=True,share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://838c457cfba44dc1bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
